<img src='./airbnb.jpg'>

## Project Recap - Where we stand

#### Overall goal
Build a (toy) Data Science project from scratch

#### Steps
- parsing airbnb pages
    - static - done
    - dynamic - done
    - [data example](https://github.com/x-technology/airbnb-analytics/blob/main/Part%201%20-%20Web%20Scraping/data_sample.csv)
- data cleaning and preprocessing
    - today
- machine learning
- building a web-extension
- (maybe some pipelining)

## Goal for today

Make clean data happen!.. With Pandas

<img src='https://res.cloudinary.com/springboard-images/image/upload/w_1080,c_limit,q_auto,f_auto,fl_lossy/wordpress/2019/08/sb-blog-data-cleaning.png'/>

Let's go!  

In [ ]:
# a typical import
import pandas as pd
import numpy as np

In [ ]:
# other stuff
import datetime
import re
import itertools
import json

## 1. Simple operations

#### a. Strings

##### split

In [ ]:
var_rooms = '8 guests · 3 bedrooms · 8 beds · 0 baths'
var_rooms.split(' · ')

##### replace

In [ ]:
var_rooms = '8 guests · 3 bedrooms · 8 beds · 0 baths'
var_rooms.replace(' · ', '**__**')

##### startswith

In [ ]:
var_str = 'random_stuff'
var_str.startswith('rand'), var_str.startswith('RAND')

#### b. Lists

##### list comprehension

In [ ]:
var_list = [1,2,3,4,5]

In [ ]:
# multiply all elements by 10
for i in var_list:
    print(10*i)

In [ ]:
[10*i for i in var_list]

##### double comprehensions

In [ ]:
var_array = [
    ['Dog', 'Cat', 'Elephant'],
    ['table', 'chair', 'couch', 'drawer']
]

In [ ]:
# comprehension with no actions - still nested
[el for el in var_array]

We want to flatten it

In [ ]:
# double comprehension
[sub_el for el in var_array for sub_el in el]

##### unique values

In [ ]:
var_list = ['a', 'a', 'b', 'c', 'e', 'e', 'x', 'x', 'x']

In [ ]:
set(var_list)

#### c. Dataframes

In [ ]:
data = pd.read_csv('../../airbnb-analytics/Part 1 - Web Scraping/Mayrhofen_AT.csv')

In [ ]:
data.head()

##### missing values

In [ ]:
data.isna().head()

In [ ]:
# we can calculate the totals
data.isna().sum()

In [ ]:
# or the non-missing totals
data.notna().sum()

##### fillna()

In [ ]:
data['refundables'].head()

In [ ]:
data['refundables'].fillna('NOPE').head()

##### replace()

In [ ]:
data['facilities'].head()

In [ ]:
data['facilities'].replace('Kitchen', 'Bathroom').head()

##### handling strings

In [ ]:
data[['rooms', 'facilities', 'header']].head()

In [ ]:
data['rooms'].str.split(' · ').head()

In [ ]:
data['facilities'].str.startswith('Kitchen').head()

##### apply

In [ ]:
data['header'].apply(str.split).head()

##### lambda function

In [ ]:
func_lambda = lambda x: 2*x

def func_normal(x):
    return 2*x

In [ ]:
print(func_lambda(4))
print(func_normal(4))

##### apply + lambda

In [ ]:
data['header'].apply(lambda x: (x, 1) if 'house' in x.split() else (x, 0)).head()

#### d. Regular expressions

In [ ]:
# we want to get a name and a family name from the corporate mail
var_str = 'Roger.Federer111@gmail.com'

In [ ]:
# .* = any symbol, any amount
re.findall(r'(.*)@(.*)', var_str)

- [a-z]+ - at least one small letter
- [A-Za-z0-9_] - one alpha-numeric symbol
- ... much more

#### e. Itertools
A Python library for looping, iterations, cross product, etc

In [ ]:
# groupby
var_list = data['facilities'].str.split().str[0].to_list()
print(var_list)

In [ ]:
[(key, len(list(group))) for key, group in itertools.groupby(sorted(var_list))]

## 2. Use-cases

In [ ]:
data_1 = pd.read_csv('../../airbnb-analytics/Part 1 - Web Scraping/Mayrhofen_AT.csv')
data_2 = pd.read_csv('../../airbnb-analytics/Part 1 - Web Scraping/Kitzbuehel_AT.csv')
data_3 = pd.read_csv('../../airbnb-analytics/Part 1 - Web Scraping/Ischgl_AT.csv')

data_1['query'] = 'Mayrhofen_AT'
data_2['query'] = 'Kitzbuehel_AT'
data_3['query'] = 'Ischgl_AT'

data = pd.concat([data_1, data_2, data_3]).reset_index(drop=True)

#### a. Split

In [ ]:
data[['listing_ratings']].head()

In [ ]:
data['listing_ratings'].str.split(r'\*\*__\*\*', expand=True)

In [ ]:
rating_columns = [
    'rating_cleanliness', 'rating_accuracy',
    'rating_communication', 'rating_location',
    'rating_check-in', 'rating_value'
]

data[rating_columns] = data['listing_ratings'].str.split('\*\*__\*\*', expand=True)

In [ ]:
data.loc[:5, rating_columns]

#### b. Merging strings

In [ ]:
# CSS has changed
data[['specialties_1', 'specialties_2']].head()

In [ ]:
# missing values
data[['specialties_1', 'specialties_2']].isna().sum()

In [ ]:
# merge and take non missing (like COALESCE in SQL)
data['specialties'] = data['specialties_1'].fillna(data['specialties_2'])

In [ ]:
# missing values
data['specialties'].isna().sum()

In [ ]:
data[['specialties_1', 'specialties_2', 'specialties']].head()

#### c. Create vocabularies

In [ ]:
# all specialties
data['specialties'].fillna('empty').str.split('\*\*__\*\*').to_list()

In [ ]:
specs_list = data['specialties'].fillna('empty').str.split('\*\*__\*\*').to_list()

In [ ]:
# double list comprehension
[sub_s for s in specs_list for sub_s in s]

In [ ]:
# unique values
set([sub_s for s in specs_list for sub_s in s])

In [ ]:
specs_uniqie = set([sub_s for s in specs_list for sub_s in s])

#### d. Filter out rare/useless values

skip:  
* superhost (separate field)
* cancellation policy (doesn't make sense)
* house rules (separate field)

shorten:
* free cancellation

In [ ]:
def process_specialties(sp_list):
    if not isinstance(sp_list, list):
        return ''
    new_list = []
    for sp in sp_list:
        if 'Free cancellation' in sp:
            new_list.append('Free cancellation')
        elif 'is a Superhost' in sp:
            continue
        elif 'Cancellation policy' in sp:
            continue
        elif 'House rules' in sp:
            continue
        else:
            new_list.append(sp)
            
    return new_list

In [ ]:
# check on one row
data['specialties'].str.split('\*\*__\*\*')[100]

In [ ]:
process_specialties(data['specialties'].str.split('\*\*__\*\*')[100])

In [ ]:
# process
data['specialties_clean'] = data['specialties'].str.split('\*\*__\*\*').apply(lambda x: process_specialties(x))

In [ ]:
data[['specialties_clean']].head()

#### e. One-Hot Encoding
- categorical (text) values are hard to use in a Machine Learning model
- it's easier to work with numbers

In [ ]:
data[['query']]

OHE  - the easiest "text -> number" transformation (embedding)

In [ ]:
pd.get_dummies(data[['query']])

In our case we have multiple values per row

In [ ]:
data_specialties = data.loc[:, ['url', 'specialties_clean']]
data_specialties.head()

In [ ]:
# all values
set([sub_s for s in data_specialties['specialties_clean'].to_list() for sub_s in s])

In [ ]:
specs_values = set([sub_s for s in data_specialties['specialties_clean'].to_list() for sub_s in s])

In [ ]:
# manual OHE
for _s in specs_values:
    data_specialties[f"specialty_{_s}"] = data_specialties['specialties_clean'].apply(lambda x: 1 if _s in x else 0)

In [ ]:
data_specialties.head()

#### f. Applying regex

In [ ]:
data[['price_per_night']].head()

##### Constructing a regex
1. There might be the first price and a "space" symbol:  
__(price ){0,1}__
2. the price is a EUR/USD symbol and at least one digit:  
__(\[€\$\]\[0-9\]+ ){0,1}__
3. then there is always the second price  
__(\[€\$\]\[0-9\]+)__
4. but we want to extract only the number, so we put EUR/USD symbol out of parentheses  
__\[€\$\](\[0-9\]+)__
5. and there's "/ night"  
6. finally  
__r'(\[€\\$\]\[0-9\]\+ ){0,1}\[€\\$\](\[0-9\]+)/ night'__  
__r'<font color='red'>(\[€\\$\]\[0-9\]\+ ){0,1}</font><font color='blue'>\[€\\$\]</font><font color='green'>(\[0-9\]+)</font>/ night'__

In [ ]:
# trying how it works
price_regex = r'([€$][0-9]+ ){0,1}[€$]([0-9]+)/ night'

print(data.loc[0, 'price_per_night'], re.findall(price_regex, data.loc[0, 'price_per_night']))
print(data.loc[136, 'price_per_night'], re.findall(price_regex, data.loc[136, 'price_per_night']))

In [ ]:
data['price_per_night'].apply(lambda x: re.findall(price_regex, x))

In [ ]:
data['price_night_EUR'] = data['price_per_night'].apply(lambda x: re.findall(price_regex, x)).str[0].str[1].astype(float)

In [ ]:
data['price_night_EUR']

## 3. Summary

### - Used methods
1. __As is__ ✔️
    - url
    - header
    - location
    - query
2. __Split__ ✂️
    - rooms
    - facilities
    - ratings
    - reviews
    - prices
    - host_feats
3. __Coalesce__ 📌
    - [name, name_alt]
    - [specialties_1, specialties_2]
    - [prices_1, prices_2]
4. __Filter and regex__ 🤏
    - prices
    - host_feats
    - house rules
5. __One Hot Encoding__ 🌡️
    - facilities
    - specialties
    - languages
    - amenities

### - Out of scope
- duplicates
- convertion and scales (USD <-> EUR, 9 PM <-> 21:00)
- typos

### - Discussion
- how to write clean Pandas code?
- use SQL if possible
- apply preprocessing while parsing

### - Look at the results
https://github.com/x-technology/airbnb-analytics/blob/main/Part%202%20-%20Data%20Cleaning/sample_clean.csv